URL: https://github.com/christophertbrown/iRep

### Instructions
#### Inputs
iRep requires high-quality draft genomes (>=75% complete, <=175 fragments/Mbp sequence, and <=2% contamination). iRep will only make use of scaffolds ≥5 Kbp in length, thus it is important to make sure that estimates of genome completion are calculated based on scaffolds that pass this threshold

I also want to select just the most complete SAG per genus

The second set of inputs are mapping files in SAM format with the --reorder flag, which should be generated by mapping DNA sequencing reads against assembled genomes using Bowtie2. In this case, provide the path to each SAM file generated for each sample

rerun on 2021-12-07 after correcting for salinity of media samples

In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP')
outdir=('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP')
matplotlib.__version__

'3.3.3'

In [2]:
SAG_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv',
                    index_col=0)
SAG_data=SAG_data[SAG_data['checkM_estimated_completeness'] >= 75]

SAGs75perc=SAG_data['name'].to_list()

MyFile=open('75perc_SAGs.txt','w')

for element in SAGs75perc:
     MyFile.write(element)
     MyFile.write('\n')
MyFile.close()
SAG_data

,X2,name,Plate,well_type,wga_cp,wgs_run_id,raw_read_count,final_assembly_length,max_contig_length,gc_content,...,sample,slope,intercept,normalized_531_value,Predicted_respiration_rate_fmol_O2perhr,lower_resp_bound,upper_resp_bound,SAG,GTDB_classification,GTDBtk_classification
1,A02,AH-135-A02,AH-135,1 cell,1.460275237,AH-135,2502502,1800737,167471,36.025194,...,AH-135-A02,0.10985,0.18027,14.25390,0.063356,0.010469,0.383414,AH-135-A02,Hel1-33-131,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...
3,A04,AH-135-A04,AH-135,1 cell,1.497029617,AH-135,3618815,1481697,223427,36.515900,...,AH-135-A04,0.10985,0.18027,8.13793,0.019320,0.003194,0.116857,AH-135-A04,Thioglobus,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
6,A07,AH-135-A07,AH-135,1 cell,1.726013961,AH-135,6300693,1141311,307768,29.599995,...,NaN,0.10985,0.18027,2.98469,0.007103,0.001165,0.043305,NaN,Unclassified,Unclassified
12,A14,AH-135-A14,AH-135,1 cell,1.673261954,AH-135,5754909,1679721,178475,34.100365,...,AH-135-A14,0.10985,0.18027,15.29750,0.077588,0.012806,0.470101,AH-135-A14,Polaribacter,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...
16,A18,AH-135-A18,AH-135,1 cell,1.618199506,AH-135,5882001,2105486,314702,35.388552,...,AH-135-A18,0.10985,0.18027,16.33200,0.094850,0.015631,0.575543,AH-135-A18,MS024-2A,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5784,M13,AH-273-M13,AH-273,1 cell,1.517333187,AH-273,3412078,1590989,127065,33.726883,...,AH-273-M13,0.10055,0.32122,34.71660,NaN,NaN,NaN,AH-273-M13,Hel1-33-131,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...
5794,N03,AH-273-N03,AH-273,1 cell,1.587519407,AH-273,3210354,1044397,166130,29.393899,...,AH-273-N03,0.10055,0.32122,11.93090,NaN,NaN,NaN,AH-273-N03,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
5803,N16,AH-273-N16,AH-273,1 cell,1.34815115,AH-273,3136144,1097658,333887,29.903030,...,AH-273-N16,0.10055,0.32122,11.69670,NaN,NaN,NaN,AH-273-N16,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
5807,N21,AH-273-N21,AH-273,1 cell,1.226552412,AH-273,3108541,2073803,192110,44.518790,...,AH-273-N21,0.10055,0.32122,29.33790,NaN,NaN,NaN,AH-273-N21,MB11C04,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...


In [3]:
#If file does not exist do the things to create it
#
if os.path.isfile('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/75perc_complete_SAGs/75perc_stats.tab')== False:        
    os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/')
    !grep -f 75perc_SAGs.txt /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/SAG_contigs.txt > 75perc_complete_SAGs/75perc_SAG_contigs.txt

    os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/75perc_complete_SAGs')

    !while read file; do cp "$file" . ; done < 75perc_SAG_contigs.txt
    !seqkit stats *.fasta -T > 75perc_stats.tab
    os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/')

In [4]:

stats=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/75perc_complete_SAGs/75perc_stats.tab',
                 sep='\t')
stats['SAG']=stats['file'].str[:10]
contigs=stats[['num_seqs', 'SAG']].copy()
contigs

,num_seqs,SAG
0,50,AH-135-A02
1,46,AH-135-A04
2,59,AH-135-A14
3,45,AH-135-A18
4,72,AH-135-B03
...,...,...
355,44,AH-709-O07
356,47,AH-709-P04
357,39,AH-709-P06
358,51,AH-709-P13


In [5]:
SAG_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv',
                    index_col=0)
SAG_data=SAG_data[SAG_data['checkM_estimated_completeness'] >= 75]
SAG_data=SAG_data.merge(contigs, how='left', on='SAG', )

SAG_data['frag_per_Mbp']=SAG_data['num_seqs']/(SAG_data['final_assembly_length']/1000000)

SAG_data=SAG_data[SAG_data['frag_per_Mbp'] <= 175]
SAG_data

,X2,name,Plate,well_type,wga_cp,wgs_run_id,raw_read_count,final_assembly_length,max_contig_length,gc_content,...,intercept,normalized_531_value,Predicted_respiration_rate_fmol_O2perhr,lower_resp_bound,upper_resp_bound,SAG,GTDB_classification,GTDBtk_classification,num_seqs,frag_per_Mbp
0,A02,AH-135-A02,AH-135,1 cell,1.460275237,AH-135,2502502,1800737,167471,36.025194,...,0.18027,14.25390,0.063356,0.010469,0.383414,AH-135-A02,Hel1-33-131,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,50.0,27.766409
1,A04,AH-135-A04,AH-135,1 cell,1.497029617,AH-135,3618815,1481697,223427,36.515900,...,0.18027,8.13793,0.019320,0.003194,0.116857,AH-135-A04,Thioglobus,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,46.0,31.045484
3,A14,AH-135-A14,AH-135,1 cell,1.673261954,AH-135,5754909,1679721,178475,34.100365,...,0.18027,15.29750,0.077588,0.012806,0.470101,AH-135-A14,Polaribacter,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,59.0,35.124881
4,A18,AH-135-A18,AH-135,1 cell,1.618199506,AH-135,5882001,2105486,314702,35.388552,...,0.18027,16.33200,0.094850,0.015631,0.575543,AH-135-A18,MS024-2A,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,45.0,21.372738
5,B03,AH-135-B03,AH-135,1 cell,1.359260806,AH-135,3832623,2566741,174985,47.661763,...,0.18027,11.48640,0.037016,0.006126,0.223657,AH-135-B03,Pseudohongiellaceae,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,72.0,28.051136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,M13,AH-273-M13,AH-273,1 cell,1.517333187,AH-273,3412078,1590989,127065,33.726883,...,0.32122,34.71660,NaN,NaN,NaN,AH-273-M13,Hel1-33-131,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,74.0,46.511949
369,N03,AH-273-N03,AH-273,1 cell,1.587519407,AH-273,3210354,1044397,166130,29.393899,...,0.32122,11.93090,NaN,NaN,NaN,AH-273-N03,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,47.0,45.002044
370,N16,AH-273-N16,AH-273,1 cell,1.34815115,AH-273,3136144,1097658,333887,29.903030,...,0.32122,11.69670,NaN,NaN,NaN,AH-273-N16,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,29.0,26.419887
371,N21,AH-273-N21,AH-273,1 cell,1.226552412,AH-273,3108541,2073803,192110,44.518790,...,0.32122,29.33790,NaN,NaN,NaN,AH-273-N21,MB11C04,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,112.0,54.007059


In [6]:
# I now need to figure out how to get a list of the most complete genome per genus. only one per value
SAG_data.sort_values(by=['GTDB_classification','checkM_estimated_completeness'], ascending = False).drop_duplicates(subset = ['GTDB_classification'], keep='first')

sag_list = list(SAG_data.sort_values(by=['GTDB_classification','checkM_estimated_completeness'], ascending = False).drop_duplicates(subset = ['GTDB_classification'], keep='first')['name'])

genus_SAGs=SAG_data[SAG_data['name'].isin(sag_list)]

genus_SAGs

MyFile=open('largest_SAGs_per_genus.txt','w')

for element in sag_list:
     MyFile.write(element)
     MyFile.write('\n')
MyFile.close()


In [7]:
#copy the files for iREP into a new folder

#!mkdir iREP_SAGs
!grep -f largest_SAGs_per_genus.txt /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/SAG_contigs.txt > largest_SAGs_per_genus_path.txt

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/iREP_SAGs/')
!while read file; do cp "$file" . ; done < ../largest_SAGs_per_genus_path.txt

In [8]:
!wc -l /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/largest_SAGs_per_genus_path.txt

77 /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/iREP/largest_SAGs_per_genus_path.txt


I think that at this point I am now at the point where I need to recruit the metagenomes to the genomes for the 78 genera that I have a SAG that fulfills the basic iREP requirements.

### Questions
1. Should I expand this to the largest genome for every genus, recruit to all SAGs? (I think that this would reduce the recruitment of the rest of the SAGs). Recruit to only the ~78 SAGs that I just identified This might result in overrecruitment of reads to these genomes. Does this matter for this analysis?


In [6]:
#!mkdir iRep_SAGs_5kb
!for file in iREP_SAGs/*.fasta; do seqkit seq -m 5000 $file -o ${file/%.fasta/_5kb.fasta}; done
!cp iREP_SAGs/*5kb.fasta iRep_SAGs_5kb/

[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remov

transform sequences (revserse, complement, extract ID...)

Usage:
  seqkit seq [flags]

Flags:
  -k, --color                     colorize sequences - to be piped into "less -R"
  -p, --complement                complement sequence, flag '-v' is recommended to switch on
      --dna2rna                   DNA to RNA
  -G, --gap-letters string        gap letters (default "- \t.")
  -h, --help                      help for seq
  -l, --lower-case                print sequences in lower case
  -M, --max-len int               only print sequences shorter than the maximum length (-1 for no limit) (default -1)
  -R, --max-qual float            only print sequences with average quality less than this limit (-1 for no limit) (default -1)
  -m, --min-len int               only print sequences longer than the minimum length (-1 for no limit) (default -1)
  -Q, --min-qual float            only print sequences with average quality qreater or equal than this limit (-1 for no limit) (default -1)
  -n, -